In [1]:
from acdesign.atmosphere import Atmosphere
from acdesign.performance.operating_point import OperatingPoint
from acdesign.airfoils.polar import UIUCPolar
from acdesign.performance.propulsion import (
    PropulsionSystem,
    FactorMotor,
    ConstantPropeller,
)
from acdesign.performance.aero import AircraftAero, WingAero, FuseAero
import plotly.express as px

import numpy as np
import pandas as pd
import plotly.graph_objects as go


clarky = UIUCPolar.local("CLARKYB")
rg15 = UIUCPolar.local("RG15C")
sa7038 = UIUCPolar.local("SA7038")
e472 = UIUCPolar.local("E472")

cell_power = 3
propulsion = PropulsionSystem(
    propeller=ConstantPropeller(0.5, 3000),
    motor=FactorMotor(0.65),
)
atm = Atmosphere.alt(0)


In [ ]:
from acdesign.aircraft.wing import Wing
from acdesign.aircraft.wing_panel import WingPanel
from acdesign.solar_wing import WingCalc
pw = 0.13



def straight_wing(nrows, ncols, section):
    b= nrows * pw + 200
    C = ncols * pw + 100

    return WingCalc(
        Wing([WingPanel.trapezoidal(b, b*C, 1)]),
        WingAero(b, b*C, [section], [0, 1])
    )
    

        
        

In [2]:
def solar_wing(nrows, ncols, section):
    b = nrows * 0.14 + 1
    C = ncols * 0.13 + 0.07
    S = b * C

    return WingAero(b, S, [section], [0, 1])


def solar_plane_aero(nrows, ncols, section):
    wing = solar_wing(nrows, ncols, section)
    fus_length = wing.b / 4
    return AircraftAero(
        wing,
        WingAero(wing.b * 0.2, wing.S * 0.2, [e472], [0, 1]),
        WingAero(wing.b * 0.1, wing.S * 0.1, [e472], [0, 1]),
        FuseAero(fus_length, 0.05),
        0.02,
        fus_length * 0.75,
    )


In [3]:
import xarray as xr


nrows = 30
ncols = 2
mass = np.linspace(4, 20, 10)

wing = solar_wing(nrows, ncols, clarky)


vmd = wing.minimize(lambda row: row.drag, atm, mass * 9.81)


vmp = (
    wing.minimize(
        lambda row: propulsion(atm, row.fs_v, row.drag),
        atm,
        mass * 9.81,
    )
    .rename(columns=dict(minVal="power"))
    .assign(mass=mass, nrows=nrows, ncols=ncols, solar_power=nrows * ncols * cell_power)
)

vmp

,wing_l,fs_v,Cl,Cd0,Cm,k,Cd,lift,drag,moment,power,mass,nrows,ncols,solar_power
5,39.24,6.524158,0.877131,0.018272,-0.068226,0.0202,0.033813,39.24,1.512688,-1.007222,30.366206,4.000000,30,2,180
6,56.68,8.337906,0.775712,0.013874,-0.075402,0.0202,0.026029,56.68,1.901925,-1.818138,48.794074,5.777778,30,2,180
3,74.12,9.378144,0.801837,0.012739,-0.075609,0.0202,0.025726,74.12,2.378082,-2.306406,68.621530,7.555556,30,2,180
4,91.56,10.437433,0.799655,0.012173,-0.075969,0.0202,0.025090,91.56,2.872781,-2.870464,92.259886,9.333333,30,2,180
5,109.00,11.428136,0.794072,0.011610,-0.076399,0.0202,0.024348,109.00,3.342137,-3.460724,117.521223,11.111111,30,2,180
5,126.44,12.354062,0.788223,0.011089,-0.076801,0.0202,0.023640,126.44,3.792057,-4.065530,144.145572,12.888889,30,2,180
4,143.88,13.215211,0.783857,0.010623,-0.077134,0.0202,0.023035,143.88,4.228143,-4.672253,171.925553,14.666667,30,2,180
5,161.32,13.862978,0.798656,0.010501,-0.076635,0.0202,0.023386,161.32,4.723686,-5.108231,201.490344,16.444444,30,2,180
4,178.76,14.621247,0.795584,0.010255,-0.076275,0.0202,0.023041,178.76,5.177106,-5.655596,232.909987,18.222222,30,2,180
4,196.20,15.375705,0.789611,0.009987,-0.075996,0.0202,0.022582,196.20,5.611066,-6.231459,265.458756,20.000000,30,2,180


In [4]:
fig = go.Figure()

fig.add_trace(go.Scatter(x=mass, y=vmd.fs_v, name="min drag speed"))
fig.add_trace(go.Scatter(x=mass, y=vmp.fs_v, name="min power speed"))

fig.add_trace(
    go.Scatter(
        x=mass,
        y=vmp.power,
        name="min power required",
        yaxis="y2",
        line=dict(dash="dash", color="black", width=2),
    )
)
fig.add_trace(
    go.Scatter(
        x=[0, 20],
        y=[cell_power * nrows * ncols, cell_power * nrows * ncols],
        name="available solar power",
        yaxis="y2",
        mode="lines",
        line=dict(dash="dot", color="gray", width=2),
    )
)

fig.update_layout(
    template="simple_white",
    title="Minimum Drag and Power Speeds vs Aircraft Mass, 30 x 2 Panel Wing",
    xaxis=dict(
        title="Mass (kg)",
        range=[4, 20],
    ),
    yaxis_title="Speed (m/s)",
    yaxis2=dict(
        title="Power Required (W)",
        overlaying="y",
        side="right",
    ),
    margin=dict(l=40, r=40, t=40, b=40),
    width=600,
    height=400,
    legend=dict(yanchor="top", y=0.3, xanchor="left", x=0.61),
)

In [5]:
vfactors = np.linspace(0.6, 1.5, 9)
rundf = (
    xr.DataArray(
        np.outer(vmd.fs_v, vfactors),
        dims=["mass", "vfactor"],
        coords={"mass": mass, "vfactor": vfactors},
    )
    .to_dataframe(name="fs_v")
    .reset_index()
    .drop(columns="vfactor")
)

df = pd.concat(
    [rundf.mass, wing(atm, rundf.fs_v, lift=rundf.mass * 9.81, mode="oto")], axis=1
)

df = df.assign(power=propulsion(atm, df.fs_v, df.drag * 1.12))


fig = go.Figure()
for _df in df.groupby("mass"):
    fig.add_trace(
        go.Scatter(
            x=_df[1].fs_v,
            y=_df[1].power,
            mode="lines",
            name=f"Mass: {_df[0]:.1f} kg",
        )
    )

fig.add_hline(
    y=nrows * ncols * cell_power,
    line=dict(color="black", width=2),
    annotation_text="Available Solar Power",
)
fig.update_layout(
    template="simple_white",
    title="Required Power vs Airspeed",
    xaxis_title="Speed (m/s)",
    yaxis_title="Required Power (W)",
    margin=dict(l=40, r=40, t=40, b=40),
    width=600,
    height=400,
    legend=dict(yanchor="top", y=1, xanchor="left", x=0.05),
)

In [6]:
from functools import partial

mass = np.linspace(1, 20, 20)
drag_multiplier = 1.12

results = []

for altitude in [0, 9000]:  # [0, 9000]:
    atm = Atmosphere.alt(altitude)
    for ncols in [2]:
        for nrows in np.arange(20, 30, 4):

            def get_power(row):
                return propulsion(atm, row.fs_v, row.drag * drag_multiplier)

            results.append(
                solar_wing(nrows, ncols, clarky)
                .minimize(
                    get_power,
                    atm,
                    mass * 9.81,
                )
                .assign(
                    altitude=altitude,
                    mass=mass,
                    nrows=nrows,
                    ncols=ncols,
                    solar_power=nrows * ncols * cell_power,
                )
            )

results = pd.concat(results).rename(columns=dict(minVal="power"))

rdf = (
    results.loc[results.power <= results.solar_power]
    .groupby(["altitude", "nrows", "ncols"])
    .apply(lambda df: df.loc[df.mass == df.mass.max()], include_groups=False)
    .reset_index()
)


colors = px.colors.qualitative.Plotly
fig = go.Figure()

for i, ((altitude, ncols), df) in enumerate(rdf.groupby(["altitude", "ncols"])):
    fig.add_trace(
        go.Scatter(
            x=df.nrows,
            y=df.mass,
            mode="lines",
            line=dict(color=colors[i % 3], dash="solid" if altitude == 0 else "dash"),
            name=f"{'Sea Level' if altitude == 0 else str(altitude) + 'm'}, {ncols} Panel Rows",
        )
    )
fig.update_layout(
    template="simple_white",
    title="Maximum Mass of Solar-Powered Aircraft",
    xaxis_title="Number of Spanwise Solar Cells",
    yaxis_title="Maximum Mass (kg)",
    width=600,
    height=400,
    legend=dict(yanchor="top", y=1, xanchor="left", x=0.01),
    margin=dict(l=40, r=40, t=40, b=40),
)

In [7]:
from acdesign.airfoils.polar import available_sections

available_sections()

{'A18',
 'A18T',
 'AVISTAR',
 'BE50',
 'BW3',
 'BW3T',
 'CLARKYB',
 'E231',
 'E374B',
 'E387A',
 'E387C',
 'E387D',
 'E423',
 'E472',
 'FALCON',
 'FX63137B',
 'GOE417A',
 'K3311',
 'LRN1007B',
 'M665',
 'M685',
 'MA409',
 'MH32',
 'MH45',
 'N6409',
 'NACA2415',
 'PT40A',
 'PT40B',
 'R140A',
 'RG14',
 'RG15B',
 'RG15C',
 'S1210',
 'S1223',
 'S4083A',
 'S4083B',
 'S5010',
 'S6062',
 'S6063',
 'S7012B',
 'S7055',
 'S8025',
 'S8036',
 'S8037',
 'S8052',
 'SA7035',
 'SA7036A',
 'SA7036B',
 'SA7038',
 'SD6060',
 'SD7003',
 'SD7032D',
 'SD7032E',
 'SD7032T',
 'SD7037A',
 'SD7037B',
 'SD7037D',
 'SD7062B',
 'SD7080',
 'SD8000',
 'SD8020',
 'SG6040',
 'SG6040T',
 'SG6041',
 'SG6041T',
 'SG6042',
 'SG6042T'}

In [8]:
from acdesign.aircraft.wing import Wing
from acdesign.aircraft.wing_panel import WingPanel

wing = Wing(
    [
        WingPanel.trapezoidal(1.5, 1.5 * 0.4, 1, 0.25),
        WingPanel.elliptical_cr(3, 0.4, 0.25),
    ]
)

print(wing)
fig = wing.plot()
fig.update_layout(
    template="simple_white",
    title="Wing Planform",
    xaxis=dict(dtick=0.1),
    yaxis=dict(dtick=0.1),
    legend=dict(yanchor="top", y=0.5, xanchor="left", x=0.01),
    margin=dict(l=40, r=40, t=40, b=40),
    width=800,
    height=300,
)


Wing(b=4.50, S=1.54, AR=13.13, smc=0.34, TR=0.00)


In [33]:
taper_results = []
trs = np.linspace(0.01, 3.0, 10)
for tr in trs:
    taper_results.append(
        Wing(
            [
                WingPanel.trapezoidal( 3, 0.5, tr),
            ]
        ).run_avl(np.linspace(0, 1, 5), np.linspace(0, 1, 50), sections=[clarky]*50)
    )


taperdf = (
    pd.concat([r[0] for r in taper_results], keys=trs)
    .reset_index()
    .rename(columns={"level_0": "TR"})
)





In [34]:
fig = go.Figure()
for tr, _df in taperdf.groupby("TR"):
    fig.add_trace(
        go.Scatter(
            x=_df.CLff,
            y=_df.CDind,
            mode="lines",
            name=f"TR={tr:.2f}",
        )
    )
fig.update_layout(
    template="simple_white",
    title="Effect of Taper Ratio on Induced Drag",
    xaxis_title="CLff",
    yaxis_title="CDind",
    margin=dict(l=40, r=40, t=40, b=40),
    width=600,
    height=400,
    legend=dict(yanchor="top", y=0.9, xanchor="left", x=0.01),
)

In [36]:
yloc = np.linspace(0, 1, 50)

fig = go.Figure()   
for tr, (loads, sloads) in zip(trs, results):
    sload = sloads[-1](yloc)
    fig.add_trace(go.Scatter(x=yloc, y=sload / sload.mean(), name=f"TR={tr:.2f}"))
fig.show()


In [40]:
airfoil_results = {}

wing = Wing(
    [
        WingPanel.trapezoidal(1.5, 1.5 * 0.4, 1, 0.25),
        WingPanel.elliptical_cr(3, 0.4, 0.25),
    ]
)

for sec in list(available_sections()):
    print(f"Running {sec}")
    section = UIUCPolar.local(sec)

    airfoil_results[sec] = wing.run_avl(
        cls=np.linspace(0, 1, 10), ylocs=np.linspace(0, 1, 50), sections=[section] * 50
    )

# loads = pd.concat({k: r[0] for k, r in results.items()} )

Running E231
Running E387A
Running PT40B


/home/td6834/source/AircraftDesign/src/acdesign/airfoils/polar.py:97: RankWarning:

Polyfit may be poorly conditioned

/home/td6834/source/AircraftDesign/src/acdesign/airfoils/polar.py:97: RankWarning:

Polyfit may be poorly conditioned

/home/td6834/source/AircraftDesign/src/acdesign/airfoils/polar.py:97: RankWarning:

Polyfit may be poorly conditioned

/home/td6834/source/AircraftDesign/src/acdesign/airfoils/polar.py:97: RankWarning:

Polyfit may be poorly conditioned



Running GOE417A


/home/td6834/source/AircraftDesign/src/acdesign/airfoils/polar.py:97: RankWarning:

Polyfit may be poorly conditioned

/home/td6834/source/AircraftDesign/src/acdesign/airfoils/polar.py:97: RankWarning:

Polyfit may be poorly conditioned

/home/td6834/source/AircraftDesign/src/acdesign/airfoils/polar.py:97: RankWarning:

Polyfit may be poorly conditioned

/home/td6834/source/AircraftDesign/src/acdesign/airfoils/polar.py:97: RankWarning:

Polyfit may be poorly conditioned

/home/td6834/source/AircraftDesign/src/acdesign/airfoils/polar.py:97: RankWarning:

Polyfit may be poorly conditioned



Running LRN1007B


...

/home/td6834/source/AircraftDesign/src/acdesign/airfoils/polar.py:97: RankWarning:

Polyfit may be poorly conditioned



Running E387C
Running SA7036B
Running SD8020
Running A18T
Running S8025
Running S8037
Running SD7037D
Running SD7032T


/home/td6834/source/AircraftDesign/src/acdesign/airfoils/polar.py:97: RankWarning:

Polyfit may be poorly conditioned

/home/td6834/source/AircraftDesign/src/acdesign/airfoils/polar.py:97: RankWarning:

Polyfit may be poorly conditioned

/home/td6834/source/AircraftDesign/src/acdesign/airfoils/polar.py:97: RankWarning:

Polyfit may be poorly conditioned

/home/td6834/source/AircraftDesign/src/acdesign/airfoils/polar.py:97: RankWarning:

Polyfit may be poorly conditioned



Running SG6041T


/home/td6834/source/AircraftDesign/src/acdesign/airfoils/polar.py:97: RankWarning:

Polyfit may be poorly conditioned

/home/td6834/source/AircraftDesign/src/acdesign/airfoils/polar.py:97: RankWarning:

Polyfit may be poorly conditioned

/home/td6834/source/AircraftDesign/src/acdesign/airfoils/polar.py:97: RankWarning:

Polyfit may be poorly conditioned

/home/td6834/source/AircraftDesign/src/acdesign/airfoils/polar.py:97: RankWarning:

Polyfit may be poorly conditioned



In [43]:
odf = (
    pd.concat({k: r[0] for k, r in airfoil_results.items()})
    .reset_index()
    .rename(columns={"level_0": "airfoil"})
)

fig = go.Figure()
for airfoil, _df in odf.groupby("airfoil"):
    fig.add_trace(
        go.Scatter(
            x=_df.CLff,
            y=_df.CDind,
            mode="lines",
            name=airfoil,
        )
    )
fig.update_layout(
    template="simple_white",
    title="Effect of Camber on Induced Drag",
    xaxis_title="CLff",
    yaxis_title="CDind",
    margin=dict(l=40, r=40, t=40, b=40),
    width=600,
    height=400,
    legend=dict(yanchor="top", y=0.9, xanchor="left", x=0.01),
)
# loads = pd.concat({k: r[0] for k, r in results.items()} )
# sloads = pd.concat({k: r[1] for k, r in results.items()} )

In [ ]:
odf

,airfoil,level_1,Surfaces,Strips,Vortices,Sref,Cref,Bref,Yref,Zref,...,Cntot,Cn'tot,CLtot,CDtot,CDvis,CDind,CLff,CDff,CYff,e
0,SG6041T,0,1.0,50.0,600.0,1.542,0.343,4.5,0.0,0.0,...,-0.0,-0.0,0.10000,0.00024,0.0,0.000242,0.10000,0.000242,0.0,1.0002
1,SG6041T,1,1.0,50.0,600.0,1.542,0.343,4.5,0.0,0.0,...,-0.0,-0.0,0.32500,0.00255,0.0,0.002553,0.32516,0.002562,0.0,1.0002
2,SG6041T,2,1.0,50.0,600.0,1.542,0.343,4.5,0.0,0.0,...,-0.0,-0.0,0.55000,0.00730,0.0,0.007299,0.55076,0.007351,0.0,1.0002
3,SG6041T,3,1.0,50.0,600.0,1.542,0.343,4.5,0.0,0.0,...,-0.0,-0.0,0.77500,0.01445,0.0,0.014453,0.77714,0.014635,0.0,1.0002
4,SG6041T,4,1.0,50.0,600.0,1.542,0.343,4.5,0.0,0.0,...,-0.0,-0.0,1.00000,0.02397,0.0,0.023973,1.00463,0.024458,0.0,1.0002
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
665,SG6042T,5,1.0,50.0,600.0,1.542,0.343,4.5,0.0,0.0,...,-0.0,-0.0,0.55556,0.00747,0.0,0.007469,0.55589,0.007476,0.0,1.0019
666,SG6042T,6,1.0,50.0,600.0,1.542,0.343,4.5,0.0,0.0,...,-0.0,-0.0,0.66667,0.01075,0.0,0.010746,0.66738,0.010771,0.0,1.0023
667,SG6042T,7,1.0,50.0,600.0,1.542,0.343,4.5,0.0,0.0,...,-0.0,-0.0,0.77778,0.01461,0.0,0.014613,0.77905,0.014673,0.0,1.0026
668,SG6042T,8,1.0,50.0,600.0,1.542,0.343,4.5,0.0,0.0,...,-0.0,-0.0,0.88889,0.01907,0.0,0.019067,0.89096,0.019187,0.0,1.0028


,,Surfaces,Strips,Vortices,Sref,Cref,Bref,Yref,Zref,Alpha,pb/2V,...,Cntot,Cn'tot,CLtot,CDtot,CDvis,CDind,CLff,CDff,CYff,e
SG6041T,0,1.0,50.0,600.0,1.542,0.343,4.5,0.0,0.0,1.08148,-0.0,...,-0.0,-0.0,0.1,0.00024,0.0,0.000242,0.1,0.000242,0.0,1.0002
M685,0,1.0,50.0,600.0,1.542,0.343,4.5,0.0,0.0,1.08148,-0.0,...,-0.0,-0.0,0.1,0.00024,0.0,0.000242,0.1,0.000242,0.0,1.0002
K3311,0,1.0,50.0,600.0,1.542,0.343,4.5,0.0,0.0,1.08148,-0.0,...,-0.0,-0.0,0.1,0.00024,0.0,0.000242,0.1,0.000242,0.0,1.0002
A18,0,1.0,50.0,600.0,1.542,0.343,4.5,0.0,0.0,1.08148,-0.0,...,-0.0,-0.0,0.1,0.00024,0.0,0.000242,0.1,0.000242,0.0,1.0002
SD6060,0,1.0,50.0,600.0,1.542,0.343,4.5,0.0,0.0,1.08148,-0.0,...,-0.0,-0.0,0.1,0.00024,0.0,0.000242,0.1,0.000242,0.0,1.0002
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
SG6040,0,1.0,50.0,600.0,1.542,0.343,4.5,0.0,0.0,1.08148,-0.0,...,-0.0,-0.0,0.1,0.00024,0.0,0.000242,0.1,0.000242,0.0,1.0002
S7012B,0,1.0,50.0,600.0,1.542,0.343,4.5,0.0,0.0,1.08148,-0.0,...,-0.0,-0.0,0.1,0.00024,0.0,0.000242,0.1,0.000242,0.0,1.0002
E387D,0,1.0,50.0,600.0,1.542,0.343,4.5,0.0,0.0,1.08148,-0.0,...,-0.0,-0.0,0.1,0.00024,0.0,0.000242,0.1,0.000242,0.0,1.0002
E387C,0,1.0,50.0,600.0,1.542,0.343,4.5,0.0,0.0,1.08148,-0.0,...,-0.0,-0.0,0.1,0.00024,0.0,0.000242,0.1,0.000242,0.0,1.0002


In [ ]:
fig = wing.plot().update_layout(
    template="simple_white",
    xaxis=dict(dtick=0.1),
    yaxis=dict(dtick=0.1),
    legend=dict(yanchor="top", y=0.7, xanchor="left", x=0.01),
)

y = np.linspace(0, wing.b / 2, 100)

fig2 = go.Figure()
for i, sload in enumerate(sloads):
    load = sload(y * 2 / wing.b)
    fig2.add_trace(
        go.Scatter(
            x=y,
            y=load / np.mean(load),
            mode="lines",
            name=f"CL={cls[i]:.2f}",
        )
    )
fig2.update_layout(
    template="simple_white",
    title="Lift Distribution",
    xaxis_title="Spanwise Location (m)",
    yaxis_title="lift_factor",
    width=800,
    height=300,
    xaxis=dict(dtick=0.1),
    margin=dict(l=40, r=40, t=40, b=40),
    legend=dict(visible=False),
)
